<img style="float: center;" src="images/CI_horizontal.png" width="600">
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>

Rayid Ghani, Frauke Kreuter, Julia Lane, Adrianne Bradford, Alex Engler, Nicolas Guetta Jeanrenaud, Graham Henke, Daniela Hochfellner, Clayton Hunter, Brian Kim, Avishek Kumar, Jonathan Morgan, Ekaterina Levitskaya, Benjamin Feder.

# Inference
----
In this notebook, we go over these main concepts:
- The use of survey weights to get accurate statistics from the sample.
- Imputation to account for missing values.

## Python Setup

As always, we start by importing any packages we need, as well as creating our connection to the database.

In [ ]:
# data manipulation
import pandas as pd
import numpy as np

# database connection
from sqlalchemy import create_engine

# statistics package for calculating survey weights
from statsmodels.stats.weightstats import DescrStatsW
import statsmodels.api as sm

# regression modeling
from sklearn.linear_model import LinearRegression

In [ ]:
# Database Connection
host = 'stuffed.adrf.info'
DB = 'appliedda'

connection_string = "postgresql://{}/{}".format(host, DB)
conn = create_engine(connection_string)

## Survey Weights

The SDR data include survey weights, which allow us to produce estimates for the total population of SEH PhD recipients. In general, survey weights are used because the sample isn't necessarily taken evenly from the population. Sometimes, researchers decide to intentionally oversample from certain subpopulations in order to make sure they have enough people from that group. Re-weighting is also done after the fact to adjust for non-response or other factors that may reduce our sample.

In this section we will compute statistics with and without sampling weights to show how the results differ and demonstrate why using weights is so important.

**Survey of Doctorate Recipients (SDR)**

As in the SDR data we are working with sub-samples of the SED population, we will need to use survey weights in our calculations.

Let's find the distribution of earnings for the SED cohort 2015. In the SDR data, we will use the variable `sdryr` (the year of first award of a U.S. PhD degree) to subset by year 2015, and we will also use the `salary` and `wtsurvy` variables.

In [ ]:
# Let's get the relevant variables from the SDR data to find the earnings 
# among the 2015 cohort

query = '''
SELECT salary, wtsurvy
FROM ncses_2019.nsf_sdr_2017
WHERE sdryr = '2015' 
'''
earnings_2015 = pd.read_sql(query,conn)

In [ ]:
# View the head of the table

earnings_2015.head()

We will use `DescrStatsW` to calculate the weighted earnings distribution. To do this, we essentially give it the variable that we want to calculate statistics for (`salary`), as well as the survey weights it should use (`wtsurvy`). The code below creates a `DescrStatsW` object, from which you obtain various weighted statistics.

In [ ]:
weighted_earnings_2015 = DescrStatsW(earnings_2015.salary, weights=earnings_2015.wtsurvy)

In [ ]:
type(weighted_earnings_2015)

To find the percentiles, we will use a built-in `pandas` function `.quantile()`.

In [ ]:
weighted_earnings_2015.quantile([.1, .25, .5, .75, .9])

Let's see what happens if we were to ignore the survey weights. You can compare this with the non-weighted estimates.

In [ ]:
earnings_2015['salary'].quantile([.1, .25, .5, .75, .9])

<font color=red><h3> Checkpoint 1: Find the distribution of earnings for 2017</h3></font>

Using the `DescrStatsW` function above, find the distribution of earnings for the cohort 2017.

In [ ]:
query = '''
SELECT salary, wtsurvy
FROM ncses_2019.nsf_sdr_2017
WHERE (INSERT VARIABLE) 
'''
earnings_2017 = 

In [ ]:
# View the first rows of the table
earnings_2017.head()

In [ ]:
# Find the weighted estimates
# Change the names of the variables

weighted_earnings_2017 = DescrStatsW(earnings_2017.VARIABLE, weights=earnings_2017.VARIABLE)

In [ ]:
# Find the percentiles
weighted_earnings_2017.quantile([.1, .25, .5, .75, .9])

To summarize, it is important to keep in mind that when using survey data (due to the specificities with which every particular survey is designed), the weights always need to be applied in order to be able to draw accurate conclusions about the general population.

### (Optional) Replicate Weights

The SDR also comes with 104 replicate weights. We won't go into too much detail on what exactly they are, but for those of you who have worked with replicate weights in the past, we will demonstrate how to use them here. 

We've separated out replicate weights in the data, so we'll need to join the main SDR table and the replicate weights table together to use them.

In [ ]:
# Let's get the relevant variables from the SDR data to find 
# the earnings among the 2015 cohort

query = '''
SELECT salary, wtsurvy, rw.*
FROM ncses_2019.nsf_sdr_2017 sdr 
JOIN ncses_2019.nsf_sdr_rw_2017 rw
ON sdr.refid = rw.refid
WHERE sdryr = '2015'
'''
earnings_rw_2015 = pd.read_sql(query,conn)

In [ ]:
# View the head of the table

earnings_rw_2015.head()

In [ ]:
# Separate out replicate weights
rw_2015 = earnings_rw_2015.iloc[:,3:]

Here, we will show an example of finding the median salary in the 2015 SDR, along with the variance of the estimate. First, we can find the median using the same method as above. 

In [ ]:
rw_earnings_2015 = DescrStatsW(earnings_rw_2015.salary, weights=earnings_rw_2015.wtsurvy)
earnings_median = rw_earnings_2015.quantile(.5)

In order to find the variance of this estimate, we can use the formula for using replicate weights as detailed in the SDR 2017 Replicate Weight User Guide (see data documentation on course website for more details). This formula is given by 

$$ v_{rep}(\hat{\theta}) = \sum^R_{r=1} 0.038461 * (\hat{\theta}_r - \hat{\theta})^2. $$

> The constant term (0.038461) is taken from the Replicate Weight User Guide.

To apply this formula quickly and easily, we will use NumPy arrays to do the operation inside the summation all at once, then add them up. The first thing we need to do is create an array of the $\hat{\theta}_r$ values, using each set of replicate weights in order to find the estimate 104 times. We'll do this with a `for` loop.

In [ ]:
# Initialize an array with NaN first
thetas = np.full(104,np.nan)

# Loop through and calculate
for i in range(104):
    thetas[i] = DescrStatsW(earnings_rw_2015.salary, weights=rw_2015.iloc[:,i]).quantile(.5)

In [ ]:
thetas

Now, we can use array operations to calculate the variance. That is, for example, if we subtract a scalar from the array, it will do the operation on each element of the array.

In [ ]:
med_var = np.sum(0.38461 * (thetas - float(earnings_median)) ** 2)

In [ ]:
# Standard error
np.sqrt(med_var)

---

## Missing Data

Sometimes, we have variables with missing (or unknown) data. Instead of dropping those values, there are methods to fill those in, in order to be able to use the data. In this example, we will look at an `age_at_dissertation` in the `nsf_sed` table, where we have a lot of missing ages. 

Keep in mind that these imputed values will be **approximations**, and must be treated as such. If you choose to impute missing values in your project or future work, you must acknowledge your process and clearly state which variables you imputed values for.

In [ ]:
# First, let's see how many missing ages we have
# Get the count of non-null age at dissertation and count of all rows

qry = '''
select count(age_at_dissertation) as age_at_diss_count, count(*) as total_count 
from ncses_2019.nsf_sed;
'''
pd.read_sql(qry, conn)

We will try to impute the unknown data using the following methods:
- Mean imputation;
- Regression imputation;
- Mode imputation (for categorical variables);
- advanced (optional): use machine learning algorithms (such as `K-nearest Neighbors`).

### Method 1. Mean Imputation

One of the simplest ways of imputing values is by taking the mean and filling it in. It's possible to do this by using the overall mean, as well as by certain subgroups. 

In [ ]:
# Let's get the SED table
query = '''
SELECT drf_id, age_at_dissertation, phdfield_name, srceprim
FROM ncses_2019.nsf_sed
'''
sed = pd.read_sql(query, conn)

In [ ]:
# Find the overall mean (could have done this in SQL too)
overall_mean = sed.age_at_dissertation.mean() 
overall_mean

We can fill the missing values with the value inside `overall_mean`, setting all the missing `age_at_dissertation` values to 34.74.

In [ ]:
complete_sed = sed.fillna(overall_mean)

We can also do this by subgroup. So, for example, we can take the mean in a certain PhD field.

In [ ]:
mean_by_field = sed.groupby('phdfield_name').mean().reset_index()
mean_by_field

We then join this back into the original DataFrame for the missing values.

### Method 2. Regression Imputation

We can also use regression to try to get more accurate values. We build a regression equation from the observations for which we know the age, then use the equation to essentially predict the missing values. This is, in effect, an extension of the mean imputation by subgroup. Here, we will use primary source of support as well as PhD field in order to impute age at dissertation.

In [ ]:
# Remove missing rows first
sed_nona = sed.dropna()

Just like in the Machine Learning notebooks, we need to use the `get_dummies` function in order to make our categorical variable into dummies.

In [ ]:
sed_dummied = pd.get_dummies(sed_nona[['phdfield_name','srceprim']], drop_first = True)

In [ ]:
sed_dummied.head()

The model creation process for a linear regression is very similar to that of the ML models when using `scikit-learn`. We create the model object, then give it the data, then use the model object to generate our predictions. The model object essentially contains all of the instructions on how to fit the model, and when we give it the data, it fits the model to that data. 

In [ ]:
# Create model object
ols = LinearRegression()

# Predictors and Outcome
predictors = sed_dummied
outcome = sed_nona.age_at_dissertation

# Fit the model
ols.fit(X = predictors, y = outcome)

Now that we've fit our model, we can find the predicted values for age.

In [ ]:
missing_x = pd.get_dummies(sed.loc[sed.age_at_dissertation.isna(),['phdfield_name','srceprim']], drop_first = True)
missing_id = sed.loc[sed.age_at_dissertation.isna(),'drf_id']

In [ ]:
missing_ages = pd.DataFrame({'drf_id':missing_id, 'age_at_dissertation':ols.predict(missing_x)})
missing_ages.head()

As before, we can join this back in to our original dataset in order to get our complete data.

<font color=red><h3> Checkpoint: Use another variable for a regression</h3></font> 

Think of another variable that can be used in the regression to predict the age at dissertation. Include it in the model, and try running it again.

## Imputation for Categorical Missing Values

### Impute using mode

For categorical variables, it doesn't make sense to try something like mean imputation, because there isn't a mean to calculate. Another method of filling in the missing values is to find the most frequent value (mode), and impute using that mode.

We use the `groupby` function to find the frequency of payment per household.

In [ ]:
# Let's get the marital status and age
query = '''
SELECT marital, age_at_dissertation
FROM ncses_2019.nsf_sed
'''
marital_status = pd.read_sql(query, conn)

First, we set the values of `marital` to `None` so that it is properly treated as missing.

In [ ]:
marital_status.loc[marital_status['marital'] == 'NA','marital'] = None

We use the `groupby` function to find the frequency of marital status categories per age.

In [ ]:
marital_count = marital_status.groupby(['age_at_dissertation','marital'])['marital'].count().reset_index(name='count')

In [ ]:
marital_count.head()

In [ ]:
# Get only the most frequent marital status values per age
marital_mode = marital_count.sort_values('count', ascending=False).drop_duplicates(['age_at_dissertation'])

marital_mode.head()

In [ ]:
# We can now drop the `count` column
marital_mode = marital_mode.drop('count',axis=1)

`Marital_mode` is now our lookup table with the most frequent value of a marital status per age.

Let's get the original table with missing values.

In [ ]:
# Get the table with missing values
query = '''
SELECT marital, age_at_dissertation
FROM ncses_2019.nsf_sed
'''
marital_status = pd.read_sql(query, conn)

# Replace NA with None object
marital_status.loc[marital_status['marital'] == 'NA','marital'] = None

We will merge the original table (`marital status`) with the look up table. This will create two columns: `marital_x` (original marital status value) and `marital_y` (marital status value from the lookup table).

In [ ]:
merged = marital_status.merge(marital_mode, on='age_at_dissertation')

Let's take a look at one age (`25`) as an example. We can see the missing values in the `marital_x` column. We are going to replace them by values in the `marital_y` column (mode that we found above).

In [ ]:
merged[merged['age_at_dissertation'] == 25]

In [ ]:
# Replace the None values in the `marital_x` column with the known values in the column `marital_y`
merged.loc[merged['marital_x'].isnull(), 'marital_x'] = merged['marital_y']

In [ ]:
# Now we can drop the `mop_y` column
merged = merged.drop('marital_y',axis=1)

Let's check at the results for the same age as we looked at before. The None values are replaced with the most frequent marital status value for that age.

In [ ]:
merged[merged['age_at_dissertation'] == 25]

### (Optional) Advanced: Using machine learning to impute values

To impute values, we can also use the machine learning algorithm called the `K-nearest Neighbors`. The principle behind it is quite simple: the missing values can be imputed by values of "closest neighbors" - as approximated by other, known, features. 

For example, if we had cases where the data on a marital status was completely missing per age, we could approximate their marital status by referring to other characteristics which could be shared by that age group (their 'closest neighbors' in terms of characteristics).

The algorithm calculates the distance between the input values (the missing values) and helps to identify the nearest possible value based on other features (such as known characteristics of the closest age group).